In [ ]:
#importing libraries
from os import path
from keras import layers
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout, Input
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import seaborn as sns
import tensorflow.compat.v1.keras.backend as K 
import tensorflow as tf 
tf.compat.v1.disable_eager_execution()

In [ ]:
#pre-processing csv file
def preprocessing(filepath):
    file=pd.read_csv(filepath)
    file=file.sample(frac=1)#shuffling the dataset
    #**************************CONVERTING CLASSES TO LIST*******************
    label_i=file.Instrument.tolist()
    label_p=file.Pitch.tolist()
    label_r=file.raga.tolist()
    encoder = LabelEncoder()
    #***************************LABEL ENCODING*******************************
    label1 = encoder.fit_transform(label_i)
    label2 = encoder.fit_transform(label_r)
    label3 = encoder.fit_transform(label_p)
    #***************************DROPPING CLASSES*****************************
    file=file.drop(columns=['filename'])
    file=file.drop(columns=['Instrument'])
    file=file.drop(columns=['Pitch'])
    file=file.drop(columns=['raga'])
    #***************************CONVERTING TO NUMPY ARRAY*********************
    file=file.to_numpy()
    file=np.reshape(file,(file.shape[0],file.shape[1],1))
    return file,label1,label2,label3 

In [ ]:
#**********************************LOADING DATASET***************************
data,label_instrument,label_raga,label_pitch=preprocessing('/home/hp/project/Music-analysis/dataset/combined_data.csv')

In [ ]:
#defining the dnn network
def rnn(file,label_instru,label_raga,label_pitch):
    #*****************for shape and size of input layer based on number of columns**********

    input1=Input(shape=(26,1))
    input=LSTM(256,return_sequences=True,name='input')(input1)
    #BRANCH FOR INSTRUMENT
    instrument=LSTM(300,return_sequences=True,name='instru_1')(input)
    instrument=LSTM(100,return_sequences=True,name='instru_2')(instrument)
    instrument=LSTM(50,name='instru_3')(instrument)
    instrument_out=Dense(6,activation='softmax',name='instrument_out')(instrument)


    #******************BRANCH FOR PITCH***********************
    pitch=LSTM(100,return_sequences=True,name='pitch_1')(input)
    pitch=LSTM(50,return_sequences=True,name='pitch_2')(pitch)
    pitch=LSTM(25,name='pitch_3')(pitch)
    pitch_out=Dense(12,activation='softmax',name='pitch_out')(pitch)


    #******************BRANCH FOR RAGA**************************************
    raga=LSTM(256,return_sequences=True,name='raga_1')(input)
    raga=LSTM(128,return_sequences=True,name='raga_2')(raga)
    raga=LSTM(64,return_sequences=True,name='raga_3')(raga)
    raga=LSTM(16,name='raga_4')(raga)
    raga_out=Dense(15,activation='softmax',name='raga_out')(raga)


    #******************DEFINTING COMBINED MODEL*******************************
    combined_mod=Model(inputs=input1,outputs=[instrument_out,pitch_out,raga_out])


    #*******************MODEL SUMMARY ****************************************
    combined_mod.summary()
    

    #*******************MODEL COMPILE ********************************
    combined_mod.compile(optimizer='adam' loss=['sparse_categorical_crossentropy',
                                                'sparse_categorical_crossentropy',
                                                'sparse_categorical_crossentropy'],
                                            metrics=['accuracy',
                                                    'accuracy',
                                                    'accuracy'])

    history=combined_mod.fit(file,{'instrument_out':label_instru,
                                    'pitch_out':label_pitch,
                                    'raga_out':label_raga},
                                    validation_split=0.4, epochs=1)
    return combined_mod,history

In [ ]:
model, history_r=rnn(data,label_instrument,label_raga,label_pitch)

In [ ]:
#saving the model as a json file with weights in h5 format
model_json = model.to_json()
with open("/home/green/Projects/dnn/model/rnn_instrument.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/green/Projects/dnn/model/rnn_instrument.h5")
print("Saved model to disk")

In [ ]:
#loading the file
json_file = open("/home/green/Projects/dnn/model/rnn_instrument.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/home/green/Projects/dnn/model/rnn_instrument.h5")
print("Loaded model from disk")
loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [91]:
################PREDICTIONS########################
predictions=model.predict(data)
y_prediction_h=predictions[0]
y_prediction_p=predictions[1]
y_prediction_r=predictions[2]

In [ ]:
for i in y_prediction_r:
    if i//1==0:
        print('Purvikalyani')
    if i//1==1:
        print('bahudari')
    if i//1==2:
        print('bauli')
    if i//1==3:
        print('behag')
    if i//1==4:
        print('brindavana saranga')
    if i//1==5:
        print('gambhira nata')
    if i//1==6:
        print('hamsadhvani')
    if i//1==7:
        print('kalyani')
    if i//1==8:
        print('kapi')
    if i//1==9:
        print('madyamavati')
    if i//1==10:
        print('natakurinji')
    if i//1==11:
        print('ragamalika ')
    if i//1==12:
        print('riti gaula')
    if i//1==13:
        print('saramathi ')
    if i//1==14:
        print('sindhubhairavi ')
    if i//1==15:
        print('yamuna kalyani')

In [ ]:
for i in y_prediction_h:
    if i//1==0:
        print('bansuri')
    if i//1==1:
        print('santoor')
    if i//1==2:
        print('sarangi')
    if i//1==3:
        print('sitar')
    if i//1==4:
        print('violin')
  

In [ ]:
for i in y_prediction_p:
    if i//1==0:
        print('A')
    if i//1==1:
        print('A#')
    if i//1==2:
        print('B')
    if i//1==3:
        print('C')
    if i//1==4:
        print('C#')
    if i//1==5:
        print('D')
    if i//1==6:
        print('D#')
    if i//1==7:
        print('E')
    if i//1==8:
        print('F')
    if i//1==9:
        print('F#')
    if i//1==10:
        print('G')
    if i//1==11:
        print('G#')